### Clase Neurona

In [7]:
class neurona(object):
    def __init__(self, f=0, c=0, dim=0):
        self.c = c
        self.f = f
        self.dim = dim
        self.w = normalize(0.5 - np.random.rand(dim).reshape(1,-1))
        self.label_winner = []
            
    def predict(self, inputs):
        return inputs @ self.w.T
    
    def fit(self, input, alfa=1):
        self.w = normalize(self.w + (alfa*input))
            
    def neuron_labeling(self, inputs, target):
        Y = inputs @ self.w.T
        self.label = target[np.argmax(Y)]
        return self.label


### Clase SOM

In [8]:
class som():
    
    def __init__(self, filas=1, columnas=1, dim=1):
        self.lista = []
        self.filas = filas
        self.columnas = columnas
        self.dim = dim
        for fila in range(self.filas):
            for columna in range(self.columnas):
                self.lista.append(neurona(f=fila, c=columna, dim=dim))
                
    def fit(self, inputs, max_epochs=1, init_radious=0, init_alfa=1):
        self.radious = init_radious
        self.alfa = init_alfa
        t = 0
        P = inputs.shape[0]
        for epoch in range(max_epochs):
            for x in inputs:
                self.alfa = init_alfa/(1.0 + float(t/P))
                i_gana, y_gana = -1, float('-inf')
                for i in range(self.filas*self.columnas):
                    y_predict = self.lista[i].predict(x.reshape(1,-1))
                    if y_predict > y_gana:
                        y_gana = y_predict
                        i_gana = i
                f_gana = int(i_gana / self.columnas)
                c_gana = i_gana % self.columnas
                
                # Conjunto de vecinas para un radious
                for f in range(f_gana - self.radious, f_gana + self.radious+1):
                    if f < 0:
                        row = self.filas + f
                    else:
                        if f > self.filas-1:
                            row = f % self.filas
                        else:
                            row = f

                    for c in range(c_gana - self.radious, c_gana + self.radious+1):
                        if c < 0:
                            column = self.columnas + c 
                        else:
                            if c > self.columnas-1:
                                column = c % self.columnas
                            else:
                                column = c
                        self.lista[(row*self.columnas) + column].fit(x.reshape(1,-1), alfa=self.alfa)
                t += 1
                if (t%1000) == 0:
                    print(t, self.radious, "  ", end='')
            if self.radious > 0:
                self.radious -= 1
                            
    def neuron_labeling(self, inputs, target):
        self.labels = []
        for i in range(self.filas*self.columnas):
            # print(X.shape, self.target.shape)
            self.labels.append(self.lista[i].neuron_labeling(inputs, target))
            # print(self.lista[i].labeling(X, target=y_deseada, etiquetado='neurona'))
 
    def predict(self, inputs):
        output_list = []
        for x in inputs:
            for i in range(self.filas*self.columnas):
                output_list.append(self.lista[i].predict(x.reshape(1,-1)))
        return np.array(output_list).reshape(inputs.shape[0], -1)    
    
    def label_predict(self, inputs):
        label_list = []
        for x in inputs:
            label_list.append(self.labels[np.argmax(self.predict(x.reshape(1,-1)))])
        return np.array(label_list).reshape(inputs.shape[0], -1)    
            
            
        
        

In [37]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler, normalize
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import accuracy_score
    

In [15]:
tabla = pd.read_hdf('MNIST_7000.h5')
data = tabla.iloc[:,:-1]
target = tabla.iloc[:,-1]

In [17]:
data_ext=np.append(data, np.ones((data.shape[0],1), dtype=float).reshape(-1,1),axis=1)
X = normalize(data_ext, axis=1)

In [19]:
data_ext.shape, X.shape

((7000, 785), (7000, 785))

In [57]:
X_train, X_test, y_train, y_test = train_test_split(X, np.array(target), test_size = 1000)

In [58]:
mapa_autorganizado = som(filas=10, columnas=6, dim=X_train.shape[1])

In [65]:
mapa_autorganizado.fit(inputs=X_train, max_epochs=10, init_radious=3, init_alfa=20)

1000 3   2000 3   3000 3   4000 3   5000 3   6000 3   7000 2   8000 2   9000 2   10000 2   11000 2   12000 2   13000 1   14000 1   15000 1   16000 1   17000 1   18000 1   19000 0   20000 0   21000 0   22000 0   23000 0   24000 0   25000 0   26000 0   27000 0   28000 0   29000 0   30000 0   31000 0   32000 0   33000 0   34000 0   35000 0   36000 0   37000 0   38000 0   39000 0   40000 0   41000 0   42000 0   43000 0   44000 0   45000 0   46000 0   47000 0   48000 0   49000 0   50000 0   51000 0   52000 0   53000 0   54000 0   55000 0   56000 0   57000 0   58000 0   59000 0   60000 0   

In [66]:
mapa_autorganizado.neuron_labeling(X_train,y_train)
lp=mapa_autorganizado.label_predict(X_test)
accuracy_score(lp,y_test)

0.701